# Mineração de dados do Github

### Bibliotecas e aqreuivos utilizados
Caso não exista o arquivo 'github_token.py' no diretorio, deve-se criar um arquivo com o seguinte conteúdo: 
 
```GITHUB_TOKEN = [PERSONAL_TOKEN]```

Esse token é gerado pelo proprio github, e será utilizado para minerar os dados do github.

Para mais informações sobre como gerar o token, acesse: https://help.github.com/articles/creating-a-personal-access-token-for-the-command-line/

In [9]:
from github import Github
from github_token import GITHUB_TOKEN
import pandas as pd

### Inicialmente criamos as instâncias da API do Github
Vamos utilizar o repositorio zulip para nossa pesquisa.

In [10]:
# criar instancia do github com o token
g = Github(GITHUB_TOKEN)

# criar uma instancia da organizacao
org = g.get_organization('zulip')

# pegar o repositorio zulip/zulip
repo = org.get_repo('zulip')

### Pegando os dados dos PR's e das issues do repositorio

In [58]:
# numero dos PR's fechados e abertos, cujo as labels são dificeis
ClosedPR = [18239, 16308, 15721, 15683, 15329, 15248, 15065, 14538, 14244, 14028, 13953, 12999, 9425, 8725]
OpenPR = [19975, 16313, 15505, 12096, 10056, 9448, 8780]

data = []

# pegar os PR's fechados
for numberPR in ClosedPR:
    pr = repo.get_pull(numberPR)

    changes = 0
    deletions = 0
    aditions = 0

    for file in pr.get_files():
        changes += file.changes
        deletions += file.deletions
        aditions += file.additions
    
    size = 0

    for comment in pr.get_comments():
        if comment.user.login != 'timabbott':
            size += len(comment.body)

    data.append(['Closed', numberPR, pr.changed_files, changes, deletions, aditions, len(pr.body), pr.created_at, pr.closed_at, pr.merged_at, pr.comments, size, pr.commits])

df = pd.DataFrame(data, columns=['Situation', 'Number', 'Changed_files', 'Changes', 'Deletion', 'Aditions', 'Size_body', 'Create_date','Closed_date', 'Merge_date', 'Comments', 'Size_comments', 'Commits'])
print(df)
# # pegar os PR's abertos
# for numberPR in OpenPR:
#     pr = repo.get_pull(numberPR)
#     print(pr.title)

# # pegar issues difíceis abertas
# for a in repo.get_issues(state='open', labels=['difficult']):
#     print(a.title)


   Situation  Number  Changed_files  Changes  Deletion  Aditions  Size_body  \
0     Closed   18239              2       70         0        70       4804   
1     Closed   16308              9      201        12       189       1437   
2     Closed   15721              3       36         3        33        742   
3     Closed   15683              2       55         0        55         98   
4     Closed   15329              7      106        21        85        211   
5     Closed   15248              5      100        33        67        243   
6     Closed   15065              5      108        16        92       1554   
7     Closed   14538              2       13         4         9        157   
8     Closed   14244              3       63        25        38        936   
9     Closed   14028              2       26         1        25        733   
10    Closed   13953              1       17         7        10        237   
11    Closed   12999             13      303        